# Tarea 2

Fecha **final** de aceptación del PR: 31 de octubre

---

El objetivo de esta tarea es implementar la integración de EDOs usando el
método de Taylor. Nos basaremos en la paquetería `TaylorSeries.jl` para
el álgebra de polinomios.
Todas las funciones necesarias deberán encontrarse en
el archivo `integracion_taylor.jl`.
Su implementación deberá pasar los tests que están en
`tests/integ_taylor.jl`, de manera similar a como lo hicimos en la Tarea 1.

El integrador deberá hacer las operaciones necesarias para obtener
los coeficientes $x_{[k]}$ de la serie de Taylor de la solución,
*en cada paso de integración*, a partir
de la condición inicial "local", o sea, al tiempo de interés.

## 0:

Familiarícense con la paquetería `TaylorSeries.jl`; para esto, lean
su documentación, en particular
[la guía del uso para una variable](https://juliadiff.org/TaylorSeries.jl/stable/userguide/#One-independent-variable).
Muchos trucos que necesitarán están escondidos ahí.

## 1:

En este ejercicio implementarán dos métodos para calcular los coeficientes
de Taylor de la o las variables dependientes en términos de la variable
independiente, tales que dichas series de Taylor satisfagan la
ecuación diferencial.
- Caso escalar: Implementen la función `coefs_taylor(f, t, u, p)`
que calculará los coeficientes $u_{[k]}$ de la expansión de Taylor
para la variable dependiente en términos de la independiente ($t$),
y regresará el desarrollo de Taylor de la solución local (`::Taylor1`).
Los argumentos de esta función son la función `f` que define
a la ecuación diferencial, la variable independiente `t::Taylor1`, la variable
dependiente `u::Taylor1`, y `p` que representa los parámetros necesarios
que se requieran en la función `f` (y que pueden ser `nothing`).
La convención para la definición de
`f` en el *caso escalar* es que tenga la forma `f(u, p, t)`), con `p`
los parámetros que sean necesarios.
- Caso vectorial: Implementen la función `coefs_taylor!(f!, t, u, du, p)`,
que usaremos cuando tenemos un *sistema* de ecuaciones diferenciales.
Los argumentos son la función
`f!` que define las ecuaciones diferenciales, la variable independiente
`t::Taylor1`, el vector (de objetos `Taylor1`) con las variables
dependientes `u`, el vector (de objetos `Taylor1`)
con el lado izquierdo de las ecuaciones diferenciales `du`, y
finalmente los parámetros necesarios representados por `p`.
La función `f!`, en el caso vectorial, usará para su definición la
convención `f!(du, u, p, t)`, y debe estar definida de tal manera que
`du` (a la salida) corresponda al lado izquierdo de
las ecuaciones diferenciales. Es decir, la `du` de entrada será modificada
por la función de manera apropiada. La función `coefs_taylor!` debe
estar implementada de tal
manera que `u` y `du` *cambien* (se actualizen) de manera adecuada,
es decir, esta función cambiará los valores de essos argumentos.
(Es por eso que, en este caso, el nombre de la función incluye `!`.)

In [1]:
using TaylorSeries, Plots

In [21]:
function coefs_taylor(f,u,p,t)
    
    u_old = copy(u)
    u_new = u + integrate(f(u_old,p,t))
    #println("Paso 0.0 : $(u_new)")
    #i = 0
    while u_new != u_old #&& i <=100
        
        u_old = u_new
        u_new = u + integrate(f(u_old,p,t))
        #i += 1
        #println("Paso $(i) : $(u_new)")
    end
    
    return u_new(t)
        
end

coefs_taylor (generic function with 1 method)

## 3:
Implementen la función `paso_integracion(u, epsilon)` con *dos métodos* (dependiendo si
estamos con una ecuación diferencial escalar o vectorial), donde se obtenga el paso
de integración $h$ a partir de los *dos últimos coeficientes* $x_{[k]}$ del desarrollo de
Taylor para las variables dependientes, multiplicado por 0.5.
(Para el caso vectorial, el paso de integración
será el menor de los pasos de integración asociados a cada variable dependiente.)
Esta función dependerá de la serie de Taylor para la variable dependiente `u`
(o del vector correspondiente), y de la tolerancia absoluta `epsilon`.

In [22]:
function paso_integracion(u,ϵ)
    N = (length(u) - 1):-1:1
    h = [0.0,0.0]
    j = 0
    for i in N
        if u[i] != 0 && j <= 2
            j += 1
            h[j] = ϵ /abs(u[i])^(1/i)
        end
        if j == 2
            break
        end
    end
    
    if h[1] < h[2]
        return h[1]
    else
        return h[2]
    end
end

paso_integracion (generic function with 1 method)

## 4:
Combinen las funciones anteriores en dos funciones, `paso_taylor` para el caso escalar y
`paso_taylor!` para el vectorial, que combine las funciones implementadas en los
ejercicios 2 y 3 adecuadamente. Estas funciones dependerán de `f`, `t`, `u`, y
`du` sólo para el caso vectorial, la tolerancia absoluta epsilon, y los parámetros `p`. Las
funciones devolverán `u` y el paso de integración `h` en el caso escalar, y en el caso vectorial
únicamente `h`, ya que el código debe ser escrito de tal manera que `u` y `du` deben
ser actualizadas/modificadas por la función (dado que son vectores, y los vectores son mutables).
Es decir, esta función deberá devolver la serie de Taylor de la solución y el paso de integración.

In [23]:
function paso_taylor(f,t,u,p,ϵ)
    u_sol = coefs_taylor(f,u,p,t)
    return u_sol, paso_integracion(u_sol,ϵ)
end

paso_taylor (generic function with 1 method)

## 5:
Escriban la función `integracion_taylor` (con dos métodos al menos) donde,
a partir de las condiciones iniciales `x0` se implementen todos los pasos necesarios
para integrar desde `t_ini` hasta `t_fin` las ecuaciones diferenciales definidas por `f`.
Los argumentos de esta función serán la función `f`, la condición inicial `x0` (escalar o
vectorial), `t_ini`, `t_fin`, el orden para los desarrollos de Taylor, la tolerancia
absoluta ϵ, y los parámetros `p` necesarios para las ecuaciones diferenciales.
Noten que si `t_ini < t_fin` la integración es "hacia adelante" en el tiempo, mientras que si
`t_ini > t_fin` la integración es "hacia atrás"; el integrador debe funcionar en ambos casos.
La función deberá devolver un vector con los tiempos calculados a cada paso de integración,
y un vector con la variable dependiente obtenida de la integración; noten que si estamos en
el caso vectorial, la salida que corresponde a los valores obtenidos de la variable dependiente
será un vector de vectores. Esta función debe ser exportada por el módulo `IntegTaylor`.

Un punto importante a notar es que el integrador debe evitar situaciones donde se tenga ciclos
infinitos, en particular, en el número de pasos de integración. Esto puede ocurrir
dado que el paso de integración es demasiado pequeño (y el tiempo final no se alcanza).
La manera de evitar esto puede ser imponiendo un número máximo de pasos de integración (que el
usuario puede cambiar), o poniendo una cota ínfima para el paso de integración. La implementación
concreta se las dejo a su criterio, pero los valores default deben permitir que
el integrador pase los tests.

In [24]:
function integracion_taylor(f, x0, t_ini, t_fin, orden, ϵ, p)
    
    u = Taylor1(x0,orden)
    t = Taylor1([t_ini,1],orden)
    u, h = paso_taylor(f,t,u,p,ϵ)
    Δt = abs(t_ini - t_fin)
    t_min = minimum([t_ini,t_fin])
    t_max = maximum([t_ini,t_fin])
    tt = t_ini
    T = [t_ini]
    X = [x0]
    hh = (-1)^(t_ini > t_fin) * h
    
    i = 1
    
    println("$(i) : $(h)")
    
    while t_min <= tt + hh <= t_max && length(T) <= 1000

        hh = (-1)^(t_ini > t_fin) * h
        tt += hh
        u = Taylor1(u(hh),orden)
        t = Taylor1([tt,1],orden)
        u, h = paso_taylor(f,t,u,p,ϵ)
        push!(T,tt)
        push!(X,u(0))
            
        i += 1
    
        println("$(i) : $(h)")
    
    end
        
    u = Taylor1(u(t_fin - tt),orden)
    t = Taylor1([t_fin,1],orden)
    u = coefs_taylor(f,u,p,t)
    push!(T,t_fin)
    push!(X,u(0))
        
    #=if t_ini > t_fin
        T = reverse(T)
        X = reverse(X)
    end=#
    
    return T, X
    
end

integracion_taylor (generic function with 1 method)

In [25]:
f(x, p, t) = x^2
exact_sol_f(t, t0, x0) = x0/(1-x0*(t-t0))

exact_sol_f (generic function with 1 method)

In [26]:
vt, vx = integracion_taylor(f, 1.0, 0.0, 0.6, 20, 1.e-20, nothing)
maximum(exact_sol_f.(vt, 0.0, 1.0) .- vx) < 1.e-14
#vt[end] == 0.6

1 : 1.0e-20
2 : 1.0e-20
3 : 1.0e-20
4 : 1.0e-20
5 : 1.0e-20
6 : 1.0e-20
7 : 1.0e-20
8 : 1.0e-20
9 : 1.0e-20
10 : 1.0e-20
11 : 1.0e-20
12 : 1.0e-20
13 : 1.0e-20
14 : 1.0e-20
15 : 1.0e-20
16 : 1.0e-20
17 : 1.0e-20
18 : 1.0e-20
19 : 1.0e-20
20 : 1.0e-20
21 : 1.0e-20
22 : 1.0e-20
23 : 1.0e-20
24 : 1.0e-20
25 : 1.0e-20
26 : 1.0e-20
27 : 1.0e-20
28 : 1.0e-20
29 : 1.0e-20
30 : 1.0e-20
31 : 1.0e-20
32 : 1.0e-20
33 : 1.0e-20
34 : 1.0e-20
35 : 1.0e-20
36 : 1.0e-20
37 : 1.0e-20
38 : 1.0e-20
39 : 1.0e-20
40 : 1.0e-20
41 : 1.0e-20
42 : 1.0e-20
43 : 1.0e-20
44 : 1.0e-20
45 : 1.0e-20
46 : 1.0e-20
47 : 1.0e-20
48 : 1.0e-20
49 : 1.0e-20
50 : 1.0e-20
51 : 1.0e-20
52 : 1.0e-20
53 : 1.0e-20
54 : 1.0e-20
55 : 1.0e-20
56 : 1.0e-20
57 : 1.0e-20
58 : 1.0e-20
59 : 1.0e-20
60 : 1.0e-20
61 : 1.0e-20
62 : 1.0e-20
63 : 1.0e-20
64 : 1.0e-20
65 : 1.0e-20
66 : 1.0e-20
67 : 1.0e-20
68 : 1.0e-20
69 : 1.0e-20
70 : 1.0e-20
71 : 1.0e-20
72 : 1.0e-20
73 : 1.0e-20
74 : 1.0e-20
75 : 1.0e-20
76 : 1.0e-20
77 : 1.0e-20
78 : 1.0

844 : 1.0e-20
845 : 1.0e-20
846 : 1.0e-20
847 : 1.0e-20
848 : 1.0e-20
849 : 1.0e-20
850 : 1.0e-20
851 : 1.0e-20
852 : 1.0e-20
853 : 1.0e-20
854 : 1.0e-20
855 : 1.0e-20
856 : 1.0e-20
857 : 1.0e-20
858 : 1.0e-20
859 : 1.0e-20
860 : 1.0e-20
861 : 1.0e-20
862 : 1.0e-20
863 : 1.0e-20
864 : 1.0e-20
865 : 1.0e-20
866 : 1.0e-20
867 : 1.0e-20
868 : 1.0e-20
869 : 1.0e-20
870 : 1.0e-20
871 : 1.0e-20
872 : 1.0e-20
873 : 1.0e-20
874 : 1.0e-20
875 : 1.0e-20
876 : 1.0e-20
877 : 1.0e-20
878 : 1.0e-20
879 : 1.0e-20
880 : 1.0e-20
881 : 1.0e-20
882 : 1.0e-20
883 : 1.0e-20
884 : 1.0e-20
885 : 1.0e-20
886 : 1.0e-20
887 : 1.0e-20
888 : 1.0e-20
889 : 1.0e-20
890 : 1.0e-20
891 : 1.0e-20
892 : 1.0e-20
893 : 1.0e-20
894 : 1.0e-20
895 : 1.0e-20
896 : 1.0e-20
897 : 1.0e-20
898 : 1.0e-20
899 : 1.0e-20
900 : 1.0e-20
901 : 1.0e-20
902 : 1.0e-20
903 : 1.0e-20
904 : 1.0e-20
905 : 1.0e-20
906 : 1.0e-20
907 : 1.0e-20
908 : 1.0e-20
909 : 1.0e-20
910 : 1.0e-20
911 : 1.0e-20
912 : 1.0e-20
913 : 1.0e-20
914 : 1.0e-20
915 : 

true

In [ ]:
vtb, vxb = integracion_taylor(f, vx[end], vt[end], vt[1], 20, 1.e-20, nothing)
maximum(exact_sol_f.(vt, vt[end], 1.0) .- vx) < 1.e-14
#vtb[end] == vt[1]
#vxb[end] ≈ vx[1]

In [95]:
# Integración "hacia adelante"
x0 = 2.0
vt, vx = integracion_taylor(flin, x0, 0.0, 0.6, 20, 1.e-20, pp)
maximum(exact_sol_flin.(vt, 0.0, x0, pp) .- vx) < 1.e-14
vt[end] == 0.6

true

In [96]:
# Integración "hacia atrás"
x0 = vx[end]
vtb, vxb = integracion_taylor(flin, x0, vt[end], vt[1], 20, 1.e-20, pp)
maximum(exact_sol_flin.(vtb, vt[end], x0, pp) .- vxb) < 1.e-14
vtb[end] == vt[1]
vxb[end] ≈ vx[1]

true

In [97]:
using Test

using TaylorSeries

@testset "Integracion EDO escalar" begin
    flin(x, p, t) = p*x
    exact_sol_flin(t, t0, x0, p) = x0 * exp(p*(t-t0))
    pp = 1.0

    # Integración "hacia adelante"
    x0 = 2.0
    vt, vx = integracion_taylor(flin, x0, 0.0, 0.6, 20, 1.e-20, pp)
    @test maximum(exact_sol_flin.(vt, 0.0, x0, pp) .- vx) < 1.e-14
    @test vt[end] == 0.6
    # Integración "hacia atrás"
    x0 = vx[end]
    vtb, vxb = integracion_taylor(flin, x0, vt[end], vt[1], 20, 1.e-20, pp)
    @test maximum(exact_sol_flin.(vtb, vt[end], x0, pp) .- vxb) < 1.e-14
    @test vtb[end] == vt[1]
    @test vxb[end] ≈ vx[1]

    pp = -2.0
    # Integración "hacia adelante"
    x0 = 2.0
    vt, vx = integracion_taylor(flin, x0, 0.0, 0.25, 20, 1.e-20, pp)
    @test maximum(exact_sol_flin.(vt, 0.0, x0, pp) .- vx) < 1.e-14
    @test vt[end] == 0.25
    # Integración "hacia atrás"
    x0 = vx[end]
    vtb, vxb = integracion_taylor(flin, x0, vt[end], vt[1], 20, 1.e-20, pp)
    @test maximum(exact_sol_flin.(vtb, vt[end], x0, pp) .- vxb) < 1.e-14
    @test vtb[end] == vt[1]
    @test vxb[end] ≈ vx[1]

    # Integración "hacia adelante"
    f(x, p, t) = x^2
    exact_sol_f(t, t0, x0) = x0/(1-x0*(t-t0))
    vt, vx = integracion_taylor(f, 1.0, 0.0, 0.6, 20, 1.e-20, nothing)
    @test maximum(exact_sol_f.(vt, 0.0, 1.0) .- vx) < 1.e-14
    @test vt[end] == 0.6
    # Integración hacia atrás
    vtb, vxb = integracion_taylor(f, vx[end], vt[end], vt[1], 20, 1.e-20, nothing)
    @test maximum(exact_sol_f.(vt, vt[end], 1.0) .- vx) < 1.e-14
    @test vtb[end] == vt[1]
    @test vxb[end] ≈ vx[1]

    # Integración "hacia adelante"
    vt, vx = integracion_taylor(f, 3.0, 0.0, 0.25, 20, 1.e-20, nothing)
    @test maximum(exact_sol_f.(vt, 0.0, 3.0) .- vx) < 1.e-14
    @test vt[end] == 0.25
    # Integración "hacia atrás"
    vtb, vxb = integracion_taylor(f, vx[end], vt[end], vt[1], 20, 1.e-20, nothing)
    @test maximum(exact_sol_f.(vt, vt[end], 3.0) .- vx) < 1.e-14
    @test vtb[end] == vt[1]
    @test vxb[end] ≈ vx[1]

    # Integración dependiente de t
    g(x, p, t) = cos(t)
    exact_sol_g(t, t0) = sin(t) - sin(t0)
    vt, vx = integracion_taylor(g, 0.0, 0.0, pi/2, 20, 1.e-20, nothing)
    @test maximum(exact_sol_g.(vt, 0.0) .- vx) < 1.e-14
    @test vt[end] ≈ pi/2
end

Integracion EDO escalar: Test Failed at In[97]:39
  Expression: maximum(exact_sol_f.(vt, 0.0, 1.0) .- vx) < 1.0e-14
   Evaluated: 5.484237660091651e-5 < 1.0e-14

Stacktrace:
 [1] macro expansion
   @ C:\Users\cocol\AppData\Local\Programs\julia-1.9.3\share\julia\stdlib\v1.9\Test\src\Test.jl:478 [inlined]
 [2] macro expansion
   @ In[97]:39 [inlined]
 [3] macro expansion
   @ C:\Users\cocol\AppData\Local\Programs\julia-1.9.3\share\julia\stdlib\v1.9\Test\src\Test.jl:1498 [inlined]
 [4] top-level scope
   @ In[97]:6
Integracion EDO escalar: Test Failed at In[97]:45
  Expression: vxb[end] ≈ vx[1]
   Evaluated: 4986.544043556556 ≈ 1.0

Stacktrace:
 [1] macro expansion
   @ C:\Users\cocol\AppData\Local\Programs\julia-1.9.3\share\julia\stdlib\v1.9\Test\src\Test.jl:478 [inlined]
 [2] macro expansion
   @ In[97]:45 [inlined]
 [3] macro expansion
   @ C:\Users\cocol\AppData\Local\Programs\julia-1.9.3\share\julia\stdlib\v1.9\Test\src\Test.jl:1498 [inlined]
 [4] top-level scope
   @ In[97]:6
Integr

LoadError: [91mSome tests did not pass: 18 passed, 4 failed, 0 errored, 0 broken.[39m

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*